# ---------------------------------------------------------
# NARRATIVE WITH SEPARATE PARAGRAPHS FOR COUNTRY PROFILES
# ---------------------------------------------------------
def narrative_separate_paragraphs(df, main_country, compare_country):
    statements = []

    # Individual country profiles (separate paragraphs)
    main_debt = df.loc[df['Metric']=="National Debt (USD)", main_country].values[0]
    main_pct_gdp = df.loc[df['Metric']=="Debt % GDP", main_country].values[0]
    main_per_capita = df.loc[df['Metric']=="Debt Per Capita", main_country].values[0]
    main_rank_debt = df.loc[df['Metric']=="National Debt (USD)", f"{main_country} Rank"].values[0]
    main_rank_gdp = df.loc[df['Metric']=="Debt % GDP", f"{main_country} Rank"].values[0]
    main_rank_pc = df.loc[df['Metric']=="Debt Per Capita", f"{main_country} Rank"].values[0]

    compare_debt = df.loc[df['Metric']=="National Debt (USD)", compare_country].values[0]
    compare_pct_gdp = df.loc[df['Metric']=="Debt % GDP", compare_country].values[0]
    compare_per_capita = df.loc[df['Metric']=="Debt Per Capita", compare_country].values[0]
    compare_rank_debt = df.loc[df['Metric']=="National Debt (USD)", f"{compare_country} Rank"].values[0]
    compare_rank_gdp = df.loc[df['Metric']=="Debt % GDP", f"{compare_country} Rank"].values[0]
    compare_rank_pc = df.loc[df['Metric']=="Debt Per Capita", f"{compare_country} Rank"].values[0]

    # Each profile is a separate paragraph
    statements.append(
        f"{main_country} has a national debt of {main_debt} (Rank {main_rank_debt}), "
        f"which is {main_pct_gdp} of its GDP (Rank {main_rank_gdp}), and a per capita debt of {main_per_capita} (Rank {main_rank_pc}).\n"
    )
    statements.append(
        f"{compare_country} has a national debt of {compare_debt} (Rank {compare_rank_debt}), "
        f"which is {compare_pct_gdp} of its GDP (Rank {compare_rank_gdp}), and a per capita debt of {compare_per_capita} (Rank {compare_rank_pc}).\n"
    )

    # Comparative statements with line breaks
    for metric in df['Metric']:
        val_main = df.loc[df['Metric']==metric, main_country].values[0]
        val_other = df.loc[df['Metric']==metric, compare_country].values[0]
        ratio_main = df.loc[df['Metric']==metric, "Ratio (main/other)"].values[0]
        ratio_inverse = df.loc[df['Metric']==metric, "Ratio (other/main)"].values[0]
        rank_main = df.loc[df['Metric']==metric, f"{main_country} Rank"].values[0]
        rank_other = df.loc[df['Metric']==metric, f"{compare_country} Rank"].values[0]
        statements.append(
            f"For {metric}, {main_country} has {val_main} (Rank {rank_main}), which is {ratio_main}× of {compare_country}'s {val_other} (Rank {rank_other}).\n"
            f"On the other hand, {compare_country} is {ratio_inverse}× bigger than {main_country}.\n"
        )

    return "\n".join(statements)

# Print narrative
print("\n--- Narrative ---")
print(narrative_separate_paragraphs(summary_df, USER_COUNTRY, HIGHLIGHT_COUNTRY))